In [1]:
# Installer les packages utiles si besoin (décommentez la ligne suivante pour le faire dans Jupyter)
# !pip install pandas pyodbc psycopg2-binary python-dotenv sqlalchemy

import os
from dotenv import load_dotenv
import pandas as pd
import pyodbc
import psycopg2
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

# Charger le .env s'il existe
load_dotenv()
print("Imports & configuration chargés.")

Imports & configuration chargés.


In [2]:
# Connexion Azure SQL
AZURE_SERVER = "carter-cash-serveur.database.windows.net"
AZURE_DATABASE = "carter_cash"
AZURE_USERNAME = "carter-cash-serveur"
AZURE_PASSWORD = "20eCoVHjWXMTUUdC"
AZURE_DRIVER = "{ODBC Driver 18 for SQL Server}"

# Connexion PostgreSQL locale Docker
PG_HOST = "localhost"
PG_PORT = "2345"
PG_DATABASE = "ada"
PG_USER = "human"
PG_PASSWORD = "pass"
print("Paramètres de connexion définis.")

Paramètres de connexion définis.


In [3]:
# Test connexion Azure SQL
try:
    cnxn_azure = pyodbc.connect(
        f"DRIVER={AZURE_DRIVER};SERVER={AZURE_SERVER};DATABASE={AZURE_DATABASE};UID={AZURE_USERNAME};PWD={AZURE_PASSWORD};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30")
    print("✅ Connexion Azure SQL OK")
    cnxn_azure.close()
except Exception as e:
    print("⛔ Connexion Azure SQL échouée :", e)

# Test connexion PostgreSQL
try:
    cnxn_pg = psycopg2.connect(
        host=PG_HOST, port=PG_PORT, dbname=PG_DATABASE, user=PG_USER, password=PG_PASSWORD)
    print("✅ Connexion PostgreSQL OK")
    cnxn_pg.close()
except Exception as e:
    print("⛔ Connexion PostgreSQL échouée :", e)

✅ Connexion Azure SQL OK
✅ Connexion PostgreSQL OK


In [4]:
def fetch_table_azure(tablename):
    cnxn_azure = pyodbc.connect(
        f"DRIVER={AZURE_DRIVER};SERVER={AZURE_SERVER};DATABASE={AZURE_DATABASE};UID={AZURE_USERNAME};PWD={AZURE_PASSWORD};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30")
    df = pd.read_sql(f"SELECT * FROM {tablename}", cnxn_azure)
    cnxn_azure.close()
    print(f"{tablename}: {df.shape[0]} lignes extraites")
    return df

tables = ["Produit", "Caracteristiques", "Dimensions", "USER_API", "DimensionsParModel"]
dfs = {}
for tab in tables:
    try:
        dfs[tab] = fetch_table_azure(tab)
    except Exception as e:
        print(f"⚠️ Lecture impossible de {tab} :", e)
print("Extraction terminée.")

Produit: 11442 lignes extraites
Caracteristiques: 11442 lignes extraites
Dimensions: 11180 lignes extraites
USER_API: 3 lignes extraites
DimensionsParModel: 191112 lignes extraites
Extraction terminée.


In [5]:
# Mappage simplifié des types pour PostgreSQL
def create_tables_pg():
    cnxn_pg = psycopg2.connect(
        host=PG_HOST, port=PG_PORT, dbname=PG_DATABASE, user=PG_USER, password=PG_PASSWORD)
    cur = cnxn_pg.cursor()
    stmts = [
        # Table Produit
        """
        DROP TABLE IF EXISTS "Produit" CASCADE;
        CREATE TABLE "Produit" (
            "ID_Produit" SERIAL PRIMARY KEY,
            "URL_Produit" VARCHAR(200),
            "Prix" INTEGER,
            "Info_generale" VARCHAR(200),
            "Descriptif" VARCHAR(200),
            "Note" VARCHAR(50),
            "Date_scrap" DATE,
            "Marque" VARCHAR(200)
        );
        """,
        # Table USER_API
        """
        DROP TABLE IF EXISTS "USER_API" CASCADE;
        CREATE TABLE "USER_API" (
            "ID_USER_API" SERIAL PRIMARY KEY,
            "username" VARCHAR(50),
            "email" VARCHAR(150),
            "full_name" VARCHAR(50),
            "hashed_password" VARCHAR(200),
            "Date_Création" DATE,
            "Date_Derniere_Connexion" DATE
        );
        """,
        # Table DimensionsParModel
        """
        DROP TABLE IF EXISTS "DimensionsParModel" CASCADE;
        CREATE TABLE "DimensionsParModel" (
            "ID_DimensionModel" SERIAL PRIMARY KEY,
            "Marque" VARCHAR(50),
            "Modele" VARCHAR(50),
            "Annee" INTEGER,
            "Finition" VARCHAR(250),
            "Largeur" INTEGER,
            "Hauteur" INTEGER,
            "Diametre" INTEGER
        );
        """,
        # Table Caracteristiques
        """
        DROP TABLE IF EXISTS "Caracteristiques" CASCADE;
        CREATE TABLE "Caracteristiques" (
            "ID_Caracteristique" SERIAL PRIMARY KEY,
            "Consommation" CHAR(1),
            "Indice_Pluie" CHAR(1),
            "Bruit" INTEGER,
            "Saisonalite" VARCHAR(50),
            "Type_Vehicule" VARCHAR(50),
            "Runflat" VARCHAR(50),
            "ID_Produit" INTEGER REFERENCES "Produit"("ID_Produit")
        );
        """,
        # Table Dimensions
        """
        DROP TABLE IF EXISTS "Dimensions" CASCADE;
        CREATE TABLE "Dimensions" (
            "ID_Dimension" SERIAL PRIMARY KEY,
            "Largeur" INTEGER,
            "Hauteur" INTEGER,
            "Diametre" INTEGER,
            "Charge" INTEGER,
            "Vitesse" CHAR(1),
            "ID_Produit" INTEGER REFERENCES "Produit"("ID_Produit")
        );
        """
    ]
    for stmt in stmts:
        cur.execute(stmt)
    cnxn_pg.commit()
    cnxn_pg.close()
    print("Tables PostgreSQL créées.")

create_tables_pg()

Tables PostgreSQL créées.


In [6]:
# Utilisez SQLAlchemy pour une insertion robuste
pg_url = f"postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DATABASE}"
engine = create_engine(pg_url)

for t in ["Produit", "USER_API", "DimensionsParModel"]:
    if not dfs.get(t) is None:
        dfs[t].to_sql(t, engine, if_exists='append', index=False)
        print(f"{t} → Insertion OK")

# Insertion des tables avec clés étrangères après "Produit"
for t in ["Caracteristiques", "Dimensions"]:
    if not dfs.get(t) is None:
        dfs[t].to_sql(t, engine, if_exists='append', index=False)
        print(f"{t} → Insertion OK")

print("Insertion terminée.")

Produit → Insertion OK
USER_API → Insertion OK
DimensionsParModel → Insertion OK
Caracteristiques → Insertion OK
Dimensions → Insertion OK
Insertion terminée.


In [7]:
with psycopg2.connect(
    host=PG_HOST, port=PG_PORT, dbname=PG_DATABASE, user=PG_USER, password=PG_PASSWORD
) as cnxn_pg:
    for t in tables:
        with cnxn_pg.cursor() as cur:
            cur.execute(f'SELECT COUNT(*) FROM "{t}"')
            count = cur.fetchone()[0]
            print(f"{t}: {count} lignes dans PostgreSQL")

Produit: 11442 lignes dans PostgreSQL
Caracteristiques: 11442 lignes dans PostgreSQL
Dimensions: 11180 lignes dans PostgreSQL
USER_API: 3 lignes dans PostgreSQL
DimensionsParModel: 191112 lignes dans PostgreSQL
